In [1]:
!pip install owlready2 rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 32.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 30.0 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24577496 sha256=aa62cea98af1a2a0c95cbc04a7a43afedb2b96c449a94aaa6c1c1b79c57c4b01
  Stored in directory: /root/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2


In [54]:
from owlready2 import *
import pandas as pd
import random

# 1. Ontology Builder
class IndustrialOntology:
    def __init__(self, ontology_path="ontology.owl"):
        self.ontology_path = ontology_path
        self.onto = get_ontology("http://example.org/industrial.owl")

    def define_classes_relations(self):
        with self.onto:
            class Produto(Thing): pass
            class Categoria(Thing): pass
            class Aplicacao(Thing): pass

            class pertence_a(Produto >> Categoria): pass
            class usado_em(Produto >> Aplicacao): pass

            class has_id(DataProperty, FunctionalProperty):
                domain = [Produto]
                range = [str]

    def save_ontology(self):
        self.onto.save(file=self.ontology_path)


# 2. Data Loader
class DataLoader:
    def __init__(self, data_path="default.csv"):
        self.data_path = data_path
        self.data = None

    def load_data(self):
        self.data = pd.read_csv(self.data_path)

    def enrich_data_with_ontology(self, ontology_obj):
        Produto = ontology_obj.onto.Produto
        Categoria = ontology_obj.onto.Categoria
        Aplicacao = ontology_obj.onto.Aplicacao

        for _, row in self.data.iterrows():
            #print(row)
            product_id = str(row["product_id"])
            category_id = str(row["category"])
            application_id = str(row["application"])

            produto = Produto(product_id)
            produto.has_id = product_id

            print(produto)

            categoria = Categoria(category_id)
            aplicacao = Aplicacao(application_id)

            produto.pertence_a.append(categoria)
            produto.usado_em.append(aplicacao)





In [53]:
import pandas as pd

df = pd.read_csv('products_new.csv')
df

,product_id,product_name,category,application,user_id
0,P1234,Motor_Eletrico,Motores,Automacao_Industrial,U5678
1,P5678,Bomba_Hidraulica,Bombas,Hidraulica,U1234
2,P91011,Sensor_de_Pressao,Sensores,Monitoramento,U5678
3,P1213,Valvula_Pneumatica,Valvulas,Controle_de_Fluxo,U91011
4,P1415,Controlador_PLC,Controladores,Automacao,U1415


In [55]:

# Step 1: Create and define ontology
ontology = IndustrialOntology()
ontology.define_classes_relations()

# Step 2: Load data and enrich ontology
data_loader = DataLoader("products_new.csv")
data_loader.load_data()
data_loader.enrich_data_with_ontology(ontology)

# Step 3: Save the updated ontology
ontology.save_ontology()


industrial.P1234
industrial.P5678
industrial.P91011
industrial.P1213
industrial.P1415


In [21]:
# 3. Recommender System
class RecommenderSystem:
    def __init__(self, ontology_wrapper, data):
        self.ontology_wrapper = ontology_wrapper
        self.ontology = ontology_wrapper.onto
        self.data = data

    def recommend_by_similarity(self, product_id):
        produto = self.ontology.search_one(has_id=product_id)
        print(produto)
        if not produto or not produto.pertence_a:
            return None

        categoria = produto.pertence_a[0]
        for p in self.ontology_wrapper.onto.Produto.instances():
            if categoria in p.pertence_a and p != produto:
                return p.name
        return None

    def recommend_by_compatibility(self, product_id):
        produto = self.ontology.search_one(has_id=product_id)
        if not produto or not produto.usado_em:
            return None

        aplicacao = produto.usado_em[0]
        for p in self.ontology_wrapper.onto.Produto.instances():
            if aplicacao in p.usado_em and p != produto:
                return p.name
        return None

    def recommend_by_profile(self, user_id):
        user_purchases = self.data[self.data["user_id"] == user_id]["product_id"].tolist()
        recommendations = set()
        for prod_id in user_purchases:
            produto = self.ontology.search_one(has_id=prod_id)
            if produto and produto.pertence_a:
                categoria = produto.pertence_a[0]
                for p in self.ontology_wrapper.onto.Produto.instances():
                    if categoria in p.pertence_a and p != produto:
                        recommendations.add(p.name)
        return list(recommendations)

In [22]:
# 4. Shopping Cart
class ShoppingCart:
    def __init__(self):
        self.items = []

    def add_item(self, product_id):
        self.items.append(product_id)

    def get_recommendations(self, recommender):
        recommendations = set()
        for item in self.items:
            print(item)
            sim = recommender.recommend_by_similarity(item)
            comp = recommender.recommend_by_compatibility(item)
            if sim:
                recommendations.add(sim)
            if comp:
                recommendations.add(comp)
        return list(recommendations)

In [23]:
recommender = RecommenderSystem(ontology, data_loader.data)


In [61]:
cart = ShoppingCart()
cart.add_item("P1234")
cart.add_item("P5678")



In [62]:
print("🛒 Recomendações para Cesta:", cart.get_recommendations(recommender))


P1234
industrial.P1234
P5678
industrial.P5678
🛒 Recomendações para Cesta: ['Motor_Eletrico', 'Bomba_Hidraulica']


In [63]:
print("📎 Similaridade:", recommender.recommend_by_similarity("P1234"))


industrial.P1234
📎 Similaridade: Motor_Eletrico


In [64]:
print("🔗 Compatibilidade:", recommender.recommend_by_compatibility("P1234"))


🔗 Compatibilidade: Motor_Eletrico


In [65]:
print("👤 Perfil do Usuário:", recommender.recommend_by_profile("U5678"))

👤 Perfil do Usuário: ['Motor_Eletrico', 'Sensor_de_Pressao']


In [57]:
for p in ontology.onto.Produto.instances():
    print("👉 Verificando produto:", p.has_id, "usado_em:", [a.name for a in p.usado_em])

👉 Verificando produto: Motor_Eletrico usado_em: ['Automacao_Industrial', 'Automacao_Industrial', 'Automacao_Industrial', 'Automacao_Industrial']
👉 Verificando produto: Bomba_Hidraulica usado_em: ['Hidraulica', 'Hidraulica', 'Hidraulica', 'Hidraulica']
👉 Verificando produto: Sensor_de_Pressao usado_em: ['Monitoramento', 'Monitoramento', 'Monitoramento', 'Monitoramento']
👉 Verificando produto: Valvula_Pneumatica usado_em: ['Controle de Fluxo', 'Controle de Fluxo', 'Controle_de_Fluxo', 'Controle_de_Fluxo']
👉 Verificando produto: Controlador_PLC usado_em: ['Automacao', 'Automacao', 'Automacao', 'Automacao']
👉 Verificando produto: P1234 usado_em: ['Automacao_Industrial', 'Automacao_Industrial', 'Automacao_Industrial', 'Automacao_Industrial', 'Automacao_Industrial']
👉 Verificando produto: P5678 usado_em: ['Hidraulica', 'Hidraulica', 'Hidraulica', 'Hidraulica', 'Hidraulica']
👉 Verificando produto: P91011 usado_em: ['Monitoramento', 'Monitoramento', 'Monitoramento', 'Monitoramento', 'Monitoram

In [51]:
print("🔍 Buscando produto:", "P1234")
produto = ontology.onto.search_one(has_id="P1234")
print("🔎 Encontrado:", produto)

🔍 Buscando produto: P1234
🔎 Encontrado: industrial.P1234
